In [1]:
%reload_ext autoreload
%autoreload 2

import os
import logging
import multiprocessing
from pathlib import Path
from tqdm import tqdm

import arviz as az
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
import scipy.stats as stats
import numpyro
from numpyro.diagnostics import hpdi

from hbmep.config import Config
# from hbmep_paper.simulator import HierarchicalBayesianModel
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)

In [2]:
# import numpyro.distributions as dist
# from hbmep.model import Baseline
# from hbmep_paper.utils.constants import HBM


# class HierarchicalBayesianModel(Baseline):
#     LINK = HBM

#     def __init__(self, config: Config):
#         super(HierarchicalBayesianModel, self).__init__(config=config)
#         self.combination_columns = self.features + [self.subject]

#     def _model(self, subject, features, intensity, response_obs=None):
#         intensity = intensity.reshape(-1, 1)
#         intensity = np.tile(intensity, (1, self.n_response))

#         feature0 = features[0].reshape(-1,)

#         n_data = intensity.shape[0]
#         n_subject = np.unique(subject).shape[0]
#         n_feature0 = np.unique(feature0).shape[0]

#         global_sigma_b = numpyro.sample("global_sigma_b", dist.HalfNormal(100))
#         global_sigma_v = numpyro.sample("global_sigma_v", dist.HalfNormal(100))

#         global_sigma_L = numpyro.sample("global_sigma_L", dist.HalfNormal(10))
#         global_sigma_H = numpyro.sample("global_sigma_H", dist.HalfNormal(500))

#         global_sigma_g_1 = numpyro.sample("global_sigma_g_1", dist.HalfNormal(5))
#         global_sigma_g_2 = numpyro.sample("global_sigma_g_2", dist.HalfNormal(5))

#         with numpyro.plate(site.n_response, self.n_response, dim=-1):
#             with numpyro.plate("n_feature0", n_feature0, dim=-2):
#                 """ Hyper-priors """
#                 mu_a = numpyro.sample(
#                     site.mu_a,
#                     dist.TruncatedNormal(50, 50, low=0)
#                 )
#                 sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

#                 sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(global_sigma_b))
#                 sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(global_sigma_v))

#                 sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(global_sigma_L))
#                 sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(global_sigma_H))

#                 sigma_g_1 = numpyro.sample("sigma_g_1", dist.HalfNormal(global_sigma_g_1))
#                 sigma_g_2 = numpyro.sample("sigma_g_2", dist.HalfNormal(global_sigma_g_2))

#                 with numpyro.plate(site.n_subject, n_subject, dim=-3):
#                     """ Priors """
#                     a = numpyro.sample(
#                         site.a,
#                         dist.TruncatedNormal(mu_a, sigma_a, low=0)
#                     )
#                     b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))
#                     v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

#                     L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
#                     H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))

#                     g_1 = numpyro.sample(site.g_1, dist.Exponential(sigma_g_1))
#                     g_2 = numpyro.sample(site.g_2, dist.Exponential(sigma_g_2))

#         """ Model """
#         mu = numpyro.deterministic(
#             site.mu,
#             L[subject, feature0]
#             + jnp.maximum(
#                 0,
#                 -1
#                 + (H[subject, feature0] + 1)
#                 / jnp.power(
#                     1
#                     + (jnp.power(1 + H[subject, feature0], v[subject, feature0]) - 1)
#                     * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0])),
#                     1 / v[subject, feature0]
#                 )
#             )
#         )
#         beta = numpyro.deterministic(
#             site.beta,
#             g_1[subject, feature0] + g_2[subject, feature0] * (1 / mu) ** 2
#         )

#         """ Observation """
#         with numpyro.plate(site.data, n_data):
#             return numpyro.sample(
#                 site.obs,
#                 dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
#                 obs=response_obs
#             )


In [3]:
import numpyro.distributions as dist
from hbmep.model import Baseline
from hbmep_paper.utils.constants import HBM


class HierarchicalBayesianModel(Baseline):
    LINK = HBM

    def __init__(self, config: Config):
        super(HierarchicalBayesianModel, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        global_sigma_b = numpyro.sample("global_sigma_b", dist.HalfNormal(100))
        global_sigma_v = numpyro.sample("global_sigma_v", dist.HalfNormal(100))

        global_sigma_L = numpyro.sample("global_sigma_L", dist.HalfNormal(10))
        global_sigma_H = numpyro.sample("global_sigma_H", dist.HalfNormal(500))

        global_sigma_g_1 = numpyro.sample("global_sigma_g_1", dist.HalfNormal(5))
        global_sigma_g_2 = numpyro.sample("global_sigma_g_2", dist.HalfNormal(5))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.Normal(50, 50)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(global_sigma_b))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(global_sigma_v))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(global_sigma_L))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(global_sigma_H))

                sigma_g_1 = numpyro.sample("sigma_g_1", dist.HalfNormal(global_sigma_g_1))
                sigma_g_2 = numpyro.sample("sigma_g_2", dist.HalfNormal(global_sigma_g_2))

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.Normal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))

                    g_1 = numpyro.sample(site.g_1, dist.Exponential(sigma_g_1))
                    g_2 = numpyro.sample(site.g_2, dist.Exponential(sigma_g_2))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[subject, feature0]
            + jnp.maximum(
                0,
                -1
                + (H[subject, feature0] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[subject, feature0], v[subject, feature0]) - 1)
                    * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0])),
                    1 / v[subject, feature0]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[subject, feature0] + g_2[subject, feature0] * (1 / mu)
        )

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


In [4]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/human/tms/fit.toml")

config = Config(toml_path=toml_path)

model = HierarchicalBayesianModel(config=config)


2023-10-06 15:14:44,944 - hbmep.config - INFO - Verifying configuration ...
2023-10-06 15:14:44,944 - hbmep.config - INFO - Success!
2023-10-06 15:14:45,184 - hbmep.model.baseline - INFO - Initialized model with hierarchical_bayesian link


In [5]:
src = "/home/vishu/data/hbmep-processed/human/tms/data.csv"
df = pd.read_csv(src)

subset = ["SCA01"]
ind = df[model.subject].isin(subset)
df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-10-06 15:14:45,253 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/tms/fit
2023-10-06 15:14:45,253 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/human/tms/fit
2023-10-06 15:14:45,254 - hbmep.dataset.core - INFO - Processing data ...
2023-10-06 15:14:45,256 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [6]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-10-06 15:14:45,335 - hbmep.model.baseline - INFO - Running inference with hierarchical_bayesian ...


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

2023-10-06 15:15:41,613 - hbmep.utils.utils - INFO - func:run_inference took: 56.28 sec


In [7]:
mcmc.print_summary(prob=.95)


                        mean       std    median      2.5%     97.5%     n_eff     r_hat
          H[0,0,0]      1.53      1.76      1.12      0.72      4.35     36.84      1.19
          H[0,0,1]      1.10      0.09      1.09      0.92      1.28    493.02      1.01
          H[0,0,2]      4.02     13.84      1.08      0.00     14.58    292.12      1.01
          H[0,0,3]      0.11      0.01      0.11      0.10      0.13    471.15      1.02
          H[0,0,4]      4.50     12.68      1.27      0.05     16.70    168.31      1.01
          H[0,0,5]      4.72     25.16      0.93      0.00     16.41    675.93      1.00
          L[0,0,0]      0.01      0.00      0.01      0.01      0.01    403.70      1.01
          L[0,0,1]      0.01      0.00      0.01      0.00      0.01    395.40      1.00
          L[0,0,2]      0.01      0.00      0.01      0.00      0.01      8.47      1.21
          L[0,0,3]      0.01      0.00      0.01      0.01      0.01    509.79      1.00
          L[0,0,4]  

In [8]:
prediction_df = model.make_prediction_dataset(df=df)

posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)

model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-10-06 15:16:16,262 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec


2023-10-06 15:16:18,939 - hbmep.utils.utils - INFO - func:predict took: 2.68 sec
2023-10-06 15:16:18,952 - hbmep.model.baseline - INFO - Rendering ...
2023-10-06 15:16:21,751 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/tms/fit/recruitment_curves.pdf
2023-10-06 15:16:21,753 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 2.81 sec
2023-10-06 15:16:21,767 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-10-06 15:16:24,415 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/tms/fit/posterior_predictive_check.pdf
2023-10-06 15:16:24,420 - hbmep.utils.utils - INFO - func:_render_predictive_check took: 2.67 sec
2023-10-06 15:16:24,421 - hbmep.utils.utils - INFO - func:render_predictive_check took: 2.67 sec


In [10]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)

In [11]:
import pickle

with open(dest, "rb") as g:
    model_, mcmc_, posterior_samples_ = pickle.load(g)


In [16]:
dest

'/home/vishu/repos/hbmep-paper/reports/human/tms/fit/inference.pkl'